In [ ]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
og_data=pd.read_csv(r'./Dataset/RTADataset.csv') #data file in same folder as project code
og_data

In [ ]:
og_data.columns

In [ ]:

data['Age_band_of_driver'].unique().tolist()
data['Vehicle_driver_relation'].unique().tolist()
data['Driving_experience'].unique().tolist()
data['Types_of_Junction'].unique().tolist()
data['Road_surface_type'].unique().tolist()
data['Light_conditions'].unique().tolist()
data['Weather_conditions'].unique().tolist()
data['Type_of_collision'].unique().tolist()
data['Vehicle_movement'].unique().tolist()
data['Cause_of_accident'].unique().tolist()
data['Accident_severity'].unique().tolist()

In [ ]:
# cleaning the dataset

#deep copy of data
data = og_data.copy()

# maintaining the only required columns in the data set
data = data.loc[:, ['Age_band_of_driver', 
                    'Vehicle_driver_relation', 
                    'Driving_experience', 
                    'Types_of_Junction', 
                    'Road_surface_type', 
                    'Light_conditions', 
                    'Weather_conditions', 
                    'Type_of_collision', 
                    'Vehicle_movement', 
                    'Cause_of_accident', 
                    'Accident_severity']]

print(data.info())

# storing the updated dataframe into the new CSV file
data.to_csv(r'../Dataset/CleanedDataset.csv', index=False)

In [ ]:
#converting categorical features to numeric
for column in data:
    print(column,': ', data[column].unique().tolist())
    
# replacing the value for accident severity
#data['Accident_severity'] = data['Accident_severity'].replace({'Slight Injury': 1, 'Serious Injury' : 2})
#data


data.Accident_severity = data.Accident_severity.map({'Slight Injury':0, 'Serious Injury':1, 'Fatal Injury': 2})

#check if unknown needs to be dropped?
filtered_data = data.dropna()

#need to map all the variables
#mapping variables that are sequentially up
#need to think on how to map variables that are not sequential
#drop nan?
#what to do with unknown?
data.Age_band_of_driver = data.Age_band_of_driver.map({'Under 18':0, '18-30':1, '31-50': 2, 'Over 51': 3, 'Unknown': 4}) 

data.Driving_experience = data.Driving_experience.map({'No Licence': 0, 'Below 1yr': 1, '1-2yr': 2 , '2-5yr': 3, '5-10yr': 4, 'Above 10yr': 5, 'unknown': 6})
data.Road_surface_type = data.Road_surface_type.map({'Earth roads': 0 , 'Gravel roads': 1, 'Asphalt roads with some distress': 2, 'Asphalt roads': 3, 'Other': 4})
data.Light_conditions = data.Light_conditions.map({'Daylight': 0, 'Darkness - lights lit': 1, 'Darkness - lights unlit': 2, 'Darkness - no lighting': 3})


#data.Vehicle_driver_relation = data.Vehicle_driver_relation.map#({'Employee', 'Unknown', 'Owner', nan, 'Other'})
#data.Types_of_Junction = data.Types_of_Junction.map# :  ['No junction', 'Y Shape', 'Crossing', 'O Shape', 'Other', 'Unknown', 'T Shape', 'X Shape', nan]
#Weather_conditions :  ['Normal', 'Raining', 'Raining and Windy', 'Cloudy', 'Other', 'Windy', 'Snow', 'Unknown', 'Fog or mist']
#Type_of_collision :  ['Collision with roadside-parked vehicles', 'Vehicle with vehicle collision', 'Collision with roadside objects', 'Collision with animals', 'Other', 'Rollover', 'Fall from vehicles', 'Collision with pedestrians', 'With Train', 'Unknown', nan]
#Vehicle_movement :  ['Going straight', 'U-Turn', 'Moving Backward', 'Turnover', 'Waiting to go', 'Getting off', 'Reversing', 'Unknown', 'Parked', 'Stopping', 'Overtaking', 'Other', 'Entering a junction', nan]
#Cause_of_accident :  ['Moving Backward', 'Overtaking', 'Changing lane to the left', 'Changing lane to the right', 'Overloading', 'Other', 'No priority to vehicle', 'No priority to pedestrian', 'No distancing', 'Getting off the vehicle improperly', 'Improper parking', 'Overspeed', 'Driving carelessly', 'Driving at high speed', 'Driving to the left', 'Unknown', 'Overturning', 'Turnover', 'Driving under the influence of drugs', 'Drunk driving']
#Accident_severity :  ['Slight Injury', 'Serious Injury', 'Fatal injury']



In [ ]:
##splitting data into training set and test set
from sklearn.model_selection import train_test_split

X = data.drop(['Accident_severity'], axis = 1)
y = data.Accident_severity

#cross validation variable
cv_num = 5

#random state
r_state=1

#80/20 test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = r_state)

In [ ]:
### Tree Model

#gradient boosting on training set with 500 trees
#effect of learning rates
#boosted_hitters = GradientBoostingRegressor(n_estimators = 500,learning_rate = 0.01, max_depth = 4, random_state = 4123)
#boosted_hitters.fit(X_train, y_train)

#function to vary learning rates; depth undefined
def variable_learn_boosting(X_train, X_test, y_train, y_test, varlearn):
    train_MSE = {}
    test_MSE = {}
    for i in varlearn:
        #classifier with rng state 4/1/23 with 500 trees and variable learning rate based on passed values
        clf = GradientBoostingRegressor(n_estimators=500, learning_rate=i, random_state=random_state)
        clf.fit(X_train, y_train)
        p = clf.predict(X_train)
        train_MSE[i] = mean_squared_error(p, y_train)
        p = clf.predict(X_test)
        test_MSE[i] = mean_squared_error(p, y_test)
    return (train_MSE, test_MSE)

###

results = variable_learn_boosting(X_train, X_test, y_train.values.ravel(), y_test.values.ravel(), np.linspace(0.01, 1, 100))

lists = sorted(results[0].items())
x, y = zip(*lists)

plt.figure(figsize=(10,5))
plt.plot(x, y, color='blue')
plt.xlabel('Learning Rates')
plt.ylabel('Corresponding Training MSE')
plt.title('MSE vs Learning Rate')
plt.grid()

###

parameters = {'learning_rate': np.linspace(0.01, 1, 100)}
clf = GridSearchCV(ensemble.GradientBoostingRegressor(random_state= r_state), parameters, cv= cv_num)
clf.fit(X= X_train, y = y_train.values.ravel())
tree_model = clf.best_estimator_
print (round(clf.best_score_,3), clf.best_params_)
p = tree_model.predict(X_test)
print("Test MSE is: " + str(round(mean_squared_error(p, y_test),3)))

###

feature_importances = tree_model.feature_importances_
feature_importances.sort()

plt.figure(figsize=(10,10))
plt.barh(X_train.columns.tolist(), feature_importances)
plt.xlabel('Features')
plt.ylabel('Importances')
plt.title("Importance of Features")
plt.grid()
plt.show()

In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import tree

from sklearn.metrics import mean_squared_error, make_scorer

from sklearn.utils import resample
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.metrics import r2_score

### Decision Tree

#parameters = {'max_depth':(2,4,6,8,10)}
parameters = {'max_depth': range(1,10)}
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

#return_train_score=True allows for train score to be tracked later on if needed
clf = GridSearchCV(tree.DecisionTreeRegressor(random_state=r_state), parameters, cv=cv_num, scoring=mse_scorer, return_train_score=True)
clf.fit(X_train, y_train)

tree_model = clf.best_estimator_
print('Optimal value of '+ str(clf.best_params_))

p = tree_model.predict(X_train)
print("Training MSE is: " + str(mean_squared_error(p, y_train)))

p = tree_model.predict(X_test)
print("Testing MSE is: " + str(mean_squared_error(p, y_test)))
###


# Set size for the plot
plt.figure(figsize=(150,10))
# Plot the tree (use filled for more clarity)
plot_tree(tree_model, filled = True, fontsize=8)

plt.show() # Display the plot.

In [ ]:
### Bagging
#bagging = BaggingRegressor(random_state=r_state)
#bagging.fit(X=X_train, y=y_train.values.ravel())
#p = bagging.predict(X_test)
#print("Test MSE is: " + str(mean_squared_error(p, y_test)))

parameters = {'n_estimators': np.arange(1, 10, 1)}
#cross validation with parameters and 5-folds
clf = GridSearchCV(ensemble.BaggingRegressor(random_state=r_state), parameters, cv=cv_num)
clf.fit(X_train, y_train.values.ravel())
bagged_tree_model = clf.best_estimator_
print ("Optimale values: ", clf.best_params_)

p = bagged_tree_model.predict(X_train)
print("Training MSE is: " + str(round(mean_squared_error(p, y_train),4)))

p = bagged_tree_model.predict(X_test)
print("Testing MSE is: " + str(round(mean_squared_error(p, y_test),4)))

feature_importances = np.mean([tree.feature_importances_ for tree in bagged_tree_model.estimators_], axis=0)
feature_importances.sort()

plt.figure(figsize=(10,10))
plt.barh(X_train.columns.tolist(), feature_importances)
plt.xlabel('Features')
plt.ylabel('Importances')
plt.title("Importance of Features")
plt.grid()
plt.show()

In [ ]:
### Random Forests
parameters = {'max_depth': range(1,10), 'n_estimators': np.arange(1, 10, 1)}
#cross validation with parameters and 5-folds
clf = GridSearchCV(ensemble.RandomForestRegressor(random_state=r_state), parameters, cv=cv_num)
clf.fit(X_train, y_train.values.ravel())
forest_model = clf.best_estimator_
print ("Optimale values: ", clf.best_params_)

p = forest_model.predict(X_train)
print("Training MSE is: " + str(round(mean_squared_error(p, y_train),4)))

p = forest_model.predict(X_test)
print("Testing MSE is: " + str(round(mean_squared_error(p, y_test),4)))


feature_importances = np.mean([tree.feature_importances_ for tree in forest_model.estimators_], axis=0)
feature_importances.sort()

plt.figure(figsize=(10,10))
plt.barh(X_train.columns.tolist(), feature_importances)
plt.xlabel('Features')
plt.ylabel('Importances')
plt.title("Importance of Features")
plt.grid()
plt.show()

###